# Model M36

|Markets      |    X  |     Y  |      W    |       CONS |
|-------------|-------|--------|-----------|------------|
|        PX   |  100  |        |   -100    | |
|        PY   |       |   100  |   -100    | |
|        PW   |       |        |    200    |       -200 |
|        PL   |  -20  |   -60  |           |        100*(1-U) |
|        PK   |  -60  |   -40  |           |        100|
|        TAX  |  -20  |     0  |           |         20|

In [1]:
using MPSGE

In [4]:
M36 = MPSGEModel()

U0 = .2

@parameters(M36, begin
    TX, 0, (description = "Proportional output tax on sector X",)
    TY, 0, (description = "Proportional output tax on sector Y",)
    TLX, 1, (description = "Ad-valorem tax on labor inputs to X",)
    TKX, 0, (description = "Ad-valorem tax on capital inputs to X",)
end)

@sectors(M36, begin
	X,	(description = "Activity level for sector X",)
	Y,	(description = "Activity level for sector Y",)
	W,	(description = "Activity level for sector W (Hicksian welfare index)",)
end)

@commodities(M36, begin
	PX,	(description = "Price index for commodity X",)
	PY,	(description = "Price index for commodity Y",)
	PL,	(description = "Price index for primary factor L (net of tax)",)
	PK,	(description = "Price index for primary factor K",)
	PW,	(description = "Price index for welfare (expenditure function)",)
end)

@consumers(M36, begin
	CONS,	(description = "Income level for consumer CONS",)
end)

@auxiliaries(M36, begin
	U,	(description = "Unemployment rate",)
end)


@production(M36, X, [s=1,t=0], begin
    @output(PX, 100, t, taxes = [Tax(CONS,TX)])
    @input(PL, 20, s, taxes = [Tax(CONS, TLX)], reference_price = 2)
    @input(PK, 60, s, taxes = [Tax(CONS, TKX)])
end)

@production(M36, Y, [s=1,t=0], begin
    @output(PY, 100, t, taxes=[Tax(CONS, TY)])
    @input(PL, 60, s)
    @input(PK, 40, s)
end)

@production(M36, W, [s=1, t=0], begin
    @output(PW, 200, t)
    @input(PX, 100, s)
    @input(PY, 100, s)
end)

@demand(M36, CONS, begin
    @final_demand(PW, 200)
    end,begin
    @endowment(PL, 80/(1-U0)-80*U/(1-U0))
    @endowment(PK, 100)
end)

@aux_constraint(M36, U,
    PL - PW
);

# Benchmark

In [6]:
set_start_value(U, U0)

fix(PW,1)

solve!(M36; cumulative_iteration_limit=0)
generate_report(M36)

Reading options file C:\Users\MPHILL~1\AppData\Local\Temp\jl_95D2.tmp
 > cumulative_iteration_limit 0
Read of options file complete.

Path 5.0.03 (Fri Jun 26 10:05:33 2020)
Written by Todd Munson, Steven Dirkse, Youngdae Kim, and Michael Ferris
Preprocessed size   : 9

Major Iteration Log
major minor  func  grad  residual    step  type prox    inorm  (label)
    0     0     1     1 0.0000e+00           I 0.0e+00 0.0e+00 (zero_profit[X)

Major Iterations. . . . 0
Minor Iterations. . . . 0
Restarts. . . . . . . . 0
Crash Iterations. . . . 0
Gradient Steps. . . . . 0
Function Evaluations. . 1
Gradient Evaluations. . 1
Basis Time. . . . . . . 0.000000
Total Time. . . . . . . 0.063000
Residual. . . . . . . . 0.000000e+00
Postsolved residual: 0.0000e+00


Solver Status: LOCALLY_SOLVED
Model Status: FEASIBLE_POINT

Row,var,value,margin
,GenericV…,Float64,Float64
1,Y,1.0,0.0
2,X,1.0,0.0
3,W,1.0,0.0
4,PL,1.0,0.0
5,PW,1.0,0.0
6,PK,1.0,0.0
7,PY,1.0,0.0
8,PX,1.0,0.0
9,CONS,200.0,0.0


# Counterfactual

In [8]:
set_value!(TLX, .25)
set_value!(TKX, .25)
set_value!(TX, 0)
set_value!(TY, 0)

solve!(M36)
generate_report(M36)

Reading options file C:\Users\MPHILL~1\AppData\Local\Temp\jl_76AD.tmp
 > cumulative_iteration_limit 10000
Read of options file complete.

Path 5.0.03 (Fri Jun 26 10:05:33 2020)
Written by Todd Munson, Steven Dirkse, Youngdae Kim, and Michael Ferris

Preprocessed size   : 9

Crash Log
major  func  diff  size  residual    step       prox   (label)
    0     0             2.9730e+01             0.0e+00 (market_clearance[PK)
    1     1     1     9 9.3746e+00  1.0e+00    0.0e+00 (income_balance[CONS)
pn_search terminated: no basis change.

Major Iteration Log
major minor  func  grad  residual    step  type prox    inorm  (label)
    0     0     2     2 9.3746e+00           I 0.0e+00 9.3e+00 (income_balance[)
    1     2     3     3 3.3608e-01  1.0e+00 SO 0.0e+00 2.7e-01 (market_clearanc)
    2     1     4     4 7.8371e-04  1.0e+00 SO 0.0e+00 6.7e-04 (market_clearanc)
    3     1     5     5 3.4958e-09  1.0e+00 SO 0.0e+00 3.0e-09 (market_clearanc)

Major Iterations. . . . 3
Minor Iterations

Row,var,value,margin
,GenericV…,Float64,Float64
1,Y,1.10646,-1.05846e-9
2,X,1.17841,-9.93822e-10
3,W,1.14187,-8.5663e-11
4,PL,1.05052,2.97665e-9
5,PW,1.0,-4.35267e-9
6,PK,1.00484,-1.01397e-9
7,PY,1.032,2.93568e-10
8,PX,0.968988,-2.11813e-10
9,CONS,228.374,2.93284e-10


# Counterfactual

In [9]:
fix(U,.2)

solve!(M36)
generate_report(M36)

Reading options file C:\Users\MPHILL~1\AppData\Local\Temp\jl_E288.tmp
 > cumulative_iteration_limit 10000
Read of options file complete.

Path 5.0.03 (Fri Jun 26 10:05:33 2020)
Written by Todd Munson, Steven Dirkse, Youngdae Kim, and Michael Ferris

Preprocessed size   : 8

Crash Log
major  func  diff  size  residual    step       prox   (label)
    0     0             2.9730e+01             0.0e+00 (market_clearance[PK)
    1     1     0     8 1.8637e+00  1.0e+00    0.0e+00 (zero_profit[Y)
pn_search terminated: no basis change.

Major Iteration Log
major minor  func  grad  residual    step  type prox    inorm  (label)
    0     0     2     2 1.8637e+00           I 0.0e+00 1.2e+00 (zero_profit[Y)
    1     1     3     3 1.6985e-03  1.0e+00 SO 0.0e+00 1.4e-03 (income_balance[)
    2     1     4     4 4.9963e-09  1.0e+00 SO 0.0e+00 2.8e-09 (market_clearanc)

Major Iterations. . . . 2
Minor Iterations. . . . 2
Restarts. . . . . . . . 0
Crash Iterations. . . . 1
Gradient Steps. . . . . 0
F

Row,var,value,margin
,GenericV…,Float64,Float64
1,Y,0.967806,-2.70312e-9
2,X,1.07779,-2.4273e-9
3,W,1.02132,-2.13532e-10
4,PL,1.17452,1.8678e-9
5,PW,1.0,-9.77187e-9
6,PK,0.898759,2.79861e-9
7,PY,1.05529,5.4925e-11
8,PX,0.947605,1.68768e-10
9,CONS,204.263,6.05382e-10
